In [1]:
import gym
import numpy as np
import tensorflow as tf

D:\Anaconda3\envs\mlcc\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\mlcc\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\mlcc\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\mlcc\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type

In [2]:
class DeepQNetwork:
    # 建立神经网络
    def _build_net(self):
        # 用来接收observation
        self.s = tf.placeholder(tf.float32, 
            [None, self.n_features], name='s')
        # 用来接收q_target的值
        self.q_target = tf.placeholder(tf.float32, 
            [None, self.n_actions], name='Q_target')
        with tf.variable_scope('eval_net'):
            # c_names(collections_names) 是在更新target_net时用到
            c_names = ['eval_net_params', 
                       tf.GraphKeys.GLOBAL_VARIABLES]
            n_l1 = 10
            w_initializer = tf.random_normal_initializer(0., 0.3)
            b_initializer = tf.constant_initializer(0.1)
        
            # eval_net的第一层
            # collections是在更新target_net参数时会用到
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1],
                    initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1],
                    initializer=b_initializer, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(self.s, w1) + b1)
            
            # eval_net的第二层
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions],
                    initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions],
                    initializer=b_initializer, collections=c_names)
                self.q_eval = tf.matmul(l1, w2) + b2
        
        with tf.variable_scope('loss'): # 求误差
            self.loss = tf.reduce_mean(tf.squared_difference(
                self.q_target, self.q_eval))
        with tf.variable_scope('train'): # 梯度下降
            self._train_op = tf.train.RMSPropOptimizer(
                self.lr).minimize(self.loss)
            
        # --------- 创建target神经网络，提供target Q--------
        self.s_ = tf.placeholder(tf.float32, [None, 
            self.n_features], name='s_') # 接收下一个observation
        with tf.variable_scope('target_net'):
            # c_names(collections_names)
            # 是在更新 target_net 参数时会用到
            c_names = ['target_net_params', 
                       tf.GraphKeys.GLOBAL_VARIABLES]
            
            # target_net 的第一层. collections 
            #是在更新 target_net 参数时会用到
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1],
                    initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1],
                    initializer=b_initializer, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(self.s_, w1) + b1)
            # target_net 的第二层. collections 
            # 是在更新 target_net 参数时会用到
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions],
                    initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions],
                    initializer=b_initializer, collections=c_names)
                self.q_next = tf.matmul(l1, w2) + b2
    
    # 初始值
    def __init__(
        self,
        n_actions,
        n_features,
        learning_rate=0.01,
        reward_decay=0.9,
        e_greedy=0.9,
        replace_target_iter=300,
        memory_size=500,
        batch_size=32,
        e_greedy_increment=None,
        output_graph=False,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy # epsilon的最大值
        # 更换target_net的步数
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size # 记忆上限
        self.batch_size = batch_size # 每次更新提取多少记忆
        self.epsilon_increment = e_greedy_increment # epsilon的增量
        # 是否开启探索模式
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max
        
        # 记录学习次数（用于判断是否更换target_net参数）
        self.learn_step_counter = 0
        
        # 初始化全0记忆[s, a, r, s_]
        self.memory = np.zeros((self.memory_size, n_features*2+2))
        
        # 创建[target_net, evaluate_net]
        self._build_net()
        
        # 替换 target net 的参数
        # 提取 target_net 的参数
        t_params = tf.get_collection('target_net_params')
        # 提取 eval_net 的参数
        e_params = tf.get_collection('eval_net_params')
        # API待查
        self.replace_target_op = [tf.assign(t, e) for t,e in zip(
            t_params, e_params)] # 更新 target_net 参数
        
        self.sess = tf.Session()
        if output_graph:
            tf.summary.FileWriter('logs/', self.sess.graph)
        
        self.sess.run(tf.global_variables_initializer())
        self.cost_his = [] # 记录所有cost变化
    
    # 存储记忆
    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        
        # 记录一条[s,a,r,s_]记录
        transition = np.hstack((s, [a,r], s_))
        
        # 总memory大小固定，若超出大小，替换旧memory
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition
        self.memory_counter += 1
    
    # 选行为
    def choose_action(self, observation):
        # 统一observation的shape(1, size_of_observation)
        observation = observation[np.newaxis, :]
        
        if np.random.uniform() < self.epsilon:
            # 让 eval_net 生成所有actions的值，并选择值最大的action
            actions_value = self.sess.run(self.q_eval, feed_dict={
                self.s: observation
            })
            action = np.argmax(actions_value)
        else:
            # 随机选择
            action = np.random.randint(0, self.n_actions)
        return action
    
    # 学习
    def learn(self):
        # 检查是否替换 target_net 参数
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.replace_target_op)
            print('\ntarget_params_replaced\n')
            
        # 从 memory 中随机抽取 batch_size 分记忆
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, 
                size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter,
                size=self.batch_size)
        batch_memory = self.memory[sample_index, :]
        
        # 获取 q_next 和 q_eval
        q_next, q_eval = self.sess.run(
            [self.q_next, self.q_eval],
            feed_dict={
                self.s_: batch_memory[:, -self.n_features:],
                self.s: batch_memory[:, :self.n_features]
            })
        
        # 下面这几步十分重要. q_next, q_eval 包含所有 action 的值,
        # 而我们需要的只是已经选择好的 action 的值, 其他的并不需要.
        # 所以我们将其他的 action 值全变成 0, 将用到的 action 误差值 反向传递回去, 作为更新凭据.
        # 这是我们最终要达到的样子, 比如 q_target - q_eval = [1, 0, 0] - [-1, 0, 0] = [2, 0, 0]
        # q_eval = [-1, 0, 0] 表示这一个记忆中有我选用过 action 0, 而 action 0 带来的 Q(s, a0) = -1, 所以其他的 Q(s, a1) = Q(s, a2) = 0.
        # q_target = [1, 0, 0] 表示这个记忆中的 r+gamma*maxQ(s_) = 1, 而且不管在 s_ 上我们取了哪个 action,
        # 我们都需要对应上 q_eval 中的 action 位置, 所以就将 1 放在了 action 0 的位置.

        # 下面也是为了达到上面说的目的, 不过为了更方面让程序运算, 达到目的的过程有点不同.
        # 是将 q_eval 全部赋值给 q_target, 这时 q_target-q_eval 全为 0,
        # 不过 我们再根据 batch_memory 当中的 action 这个 column 来给 q_target 中的对应的 memory-action 位置来修改赋值.
        # 使新的赋值为 reward + gamma * maxQ(s_), 这样 q_target-q_eval 就可以变成我们所需的样子.
        # 具体在下面还有一个举例说明.
        
        q_target = q_eval.copy()
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        eval_act_index = batch_memory[:, self.n_features].astype(int)
        reward = batch_memory[:, self.n_features + 1]
        
        q_target[batch_index, eval_act_index] = \
        reward + self.gamma*np.max(q_next, axis=1)
        
        """
        假如在这个 batch 中, 我们有2个提取的记忆, 根据每个记忆可以生产3个 action 的值:
        q_eval =
        [[1, 2, 3],
         [4, 5, 6]]

        q_target = q_eval =
        [[1, 2, 3],
         [4, 5, 6]]

        然后根据 memory 当中的具体 action 位置来修改 q_target 对应 action 上的值:
        比如在:
            记忆 0 的 q_target 计算值是 -1, 而且我用了 action 0;
            记忆 1 的 q_target 计算值是 -2, 而且我用了 action 2:
        q_target =
        [[-1, 2, 3],
         [4, 5, -2]]

        所以 (q_target - q_eval) 就变成了:
        [[(-1)-(1), 0, 0],
         [0, 0, (-2)-(6)]]

        最后我们将这个 (q_target - q_eval) 当成误差, 反向传递会神经网络.
        所有为 0 的 action 值是当时没有选择的 action, 之前有选择的 action 才有不为0的值.
        我们只反向传递之前选择的 action 的值,
        """
        # 训练 eval_net
        _, self.cost = self.sess.run([self._train_op, self.loss],
            feed_dict={self.s: batch_memory[:, :self.n_features],
            self.q_target: q_target})
        self.cost_his.append(self.cost) # 记录cost误差
        
        # 逐渐增加 epsilon，降低行为的随机性
        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1
    
    # 看看学习效果（可选）
    def plot_cost(self):
        import matplotlib.pyplot as plt
        plt.plot(np.arange(len(self.cost_his)), self.cost_his)
        plt.ylabel('Cost')
        plt.xlabel('training steps')
        plt.show()

In [3]:
env = gym.make('CartPole-v0') # 定义使用gym库中的环境

In [4]:
print(env.action_space) # 查看这个环境中可用的 action 有多少个
print(env.observation_space)    # 查看这个环境中可用的 state 的 observation 有多少个
print(env.observation_space.high)   # 查看 observation 最高取值
print(env.observation_space.low)    # 查看 observation 最低取值

Discrete(2)
Box(4,)
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


In [5]:
# 定义使用 DQN 的算法
RL = DeepQNetwork(n_actions=env.action_space.n,
                  n_features=env.observation_space.shape[0],
                  learning_rate=0.01, e_greedy=0.9,
                  replace_target_iter=100, memory_size=2000,
                  e_greedy_increment=0.0008,)

total_steps = 0 # 记录步数

W0729 16:47:01.366421 14424 deprecation.py:506] From D:\Anaconda3\envs\mlcc\lib\site-packages\tensorflow\python\training\rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
for i_episode in range(100):

    # 获取回合 i_episode 第一个 observation
    observation = env.reset()
    ep_r = 0
    while True:
        env.render()    # 刷新环境

        action = RL.choose_action(observation)  # 选行为

        observation_, reward, done, info = env.step(action) # 获取下一个 state

        x, x_dot, theta, theta_dot = observation_   # 细分开, 为了修改原配的 reward

        # x 是车的水平位移, 所以 r1 是车越偏离中心, 分越少
        # theta 是棒子离垂直的角度, 角度越大, 越不垂直. 所以 r2 是棒越垂直, 分越高

        x, x_dot, theta, theta_dot = observation_
        r1 = (env.x_threshold - abs(x))/env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta))/env.theta_threshold_radians - 0.5
        reward = r1 + r2   # 总 reward 是 r1 和 r2 的结合, 既考虑位置, 也考虑角度, 这样 DQN 学习更有效率

        # 保存这一组记忆
        RL.store_transition(observation, action, reward, observation_)

        if total_steps > 1000:
            RL.learn()  # 学习

        ep_r += reward
        if done:
            print('episode: ', i_episode,
                  'ep_r: ', round(ep_r, 2),
                  ' epsilon: ', round(RL.epsilon, 2))
            break

        observation = observation_
        total_steps += 1
# 最后输出 cost 曲线
RL.plot_cost()

episode:  0 ep_r:  3.14  epsilon:  0
episode:  1 ep_r:  8.18  epsilon:  0
episode:  2 ep_r:  4.78  epsilon:  0
episode:  3 ep_r:  4.08  epsilon:  0
episode:  4 ep_r:  1.65  epsilon:  0
episode:  5 ep_r:  11.49  epsilon:  0
episode:  6 ep_r:  5.1  epsilon:  0
episode:  7 ep_r:  6.96  epsilon:  0
episode:  8 ep_r:  18.64  epsilon:  0


KeyboardInterrupt: 